In [43]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import pandas as pd
import numpy as np

from dhdrnet.util import DATA_DIR, ROOT_DIR

figdir = ROOT_DIR / "figures"

In [45]:
import re

df = (
    pd.read_csv(
        ROOT_DIR / "precomputed_data" / "store_rmse_2020-10-26.csv", index_col=0
    )
    .drop(columns="ev1")
    .rename(columns={"ev2": "ev"})
)
test_names = pd.read_csv(ROOT_DIR / "precomputed_data" / "test_current.csv")

# p = re.compile(r"([0-9.]+)")
# mask = df["metric"] == "perceptual"
# df.loc[mask, "score"] = df.loc[mask, "score"].transform(lambda s: p.search(s).group(0))
# df["score"] = pd.to_numeric(df["score"])
df = df.set_index("name")
df = df.loc[df.index.intersection(test_names["names"])]
df = df.reset_index()

In [46]:
df.loc[df["metric"] == "rmse", "score"] =  df.loc[df["metric"] == "rmse", "score"].apply(np.sqrt)
df

,name,score,metric,ev
0,0006_20160721_170707_736,34.957907,rmse,-3.00
1,0006_20160721_170707_736,17.259895,psnr,-3.00
2,0006_20160721_170707_736,0.900156,ssim,-3.00
3,0006_20160721_170707_736,35.616869,rmse,-2.75
4,0006_20160721_170707_736,17.097689,psnr,-2.75
...,...,...,...,...
8444731,c483_20150901_221141_886,18.659113,psnr,5.75
8444732,c483_20150901_221141_886,0.877539,ssim,5.75
8444733,c483_20150901_221141_886,34.653282,rmse,6.00
8444734,c483_20150901_221141_886,17.335916,psnr,6.00


In [47]:
grouped = df.set_index("name").groupby(["metric"])
grouped.describe()["score"][["mean", "std", "min", "max"]]

,mean,std,min,max
metric,,,,
psnr,18.004786,5.801651,7.166248,50.651614
rmse,39.081830,23.138918,0.748100,111.744920
ssim,0.769130,0.231513,0.045940,0.998201


In [50]:
idxmins = (
    df[(df["metric"] == "perceptual") | (df["metric"] == "rmse")]
    .groupby(["name", "metric"])["score"]
    .idxmin()
)
best_scores_min = df.loc[idxmins]
best_scores_min
# this is the minimum score for each metric for each image
# unfortunately we want min for mse and perceptual
# and maxes for psnr, ssim

,name,score,metric,ev
78,0006_20160721_170707_736,11.903806,rmse,3.75
11745,0006_20160721_175239_909,9.563386,rmse,4.00
23409,0006_20160721_181525_062,12.342240,rmse,4.00
35067,0006_20160722_100126_028,9.298876,rmse,3.50
46686,0006_20160722_100951_201,8.976312,rmse,-0.50
...,...,...,...,...
8386446,c483_20150901_173210_443,14.053371,rmse,-0.50
8398146,c483_20150901_194029_165,20.143886,rmse,2.75
8409813,c483_20150901_195108_436,19.015955,rmse,3.00
8421483,c483_20150901_214630_530,8.471777,rmse,3.50


In [51]:
idxmaxs = (
    df[(df["metric"] == "psnr") | (df["metric"] == "ssim")]
    .groupby(["name", "metric"])["score"]
    .idxmax()
)
best_scores_max = df.loc[idxmaxs]
best_scores_max

,name,score,metric,ev
79,0006_20160721_170707_736,26.617087,psnr,3.75
59,0006_20160721_170707_736,0.975438,ssim,2.00
11746,0006_20160721_175239_909,28.518569,psnr,4.00
11747,0006_20160721_175239_909,0.965298,ssim,4.00
23410,0006_20160721_181525_062,26.302924,psnr,4.00
...,...,...,...,...
8409815,c483_20150901_195108_436,0.971811,ssim,3.00
8421484,c483_20150901_214630_530,29.571314,psnr,3.50
8421488,c483_20150901_214630_530,0.970954,ssim,3.75
8433148,c483_20150901_221141_886,31.740585,psnr,3.50


In [52]:
best_scores_min.groupby("metric").describe()["score"][["mean", "std"]]

,mean,std
metric,,
rmse,10.717247,5.932719


In [53]:
best_scores_max.groupby("metric").describe()["score"][["mean", "std"]]

,mean,std
metric,,
psnr,29.075697,5.774687
ssim,0.973383,0.014968


In [54]:
df.loc[df["ev"] == 3.5].groupby("metric").describe()["score"][["mean", "std"]]

,mean,std
metric,,
psnr,26.724160,5.659924
rmse,14.507083,10.169380
ssim,0.961232,0.023184


In [17]:
df.loc[df["ev"] == 3.5].groupby("metric").describe()["score"][["mean", "std"]].to_latex(
    figdir / "fixed_choice.tex"
)

In [57]:
cdf = pd.read_csv(ROOT_DIR / "precomputed_data" / "store_updown_2020-10-26.csv")
cdf.loc[cdf["metric"] == "rmse", "score"] =  cdf.loc[cdf["metric"] == "rmse", "score"].apply(np.sqrt)
cdf

,name,metric,ev,score
0,0030_20151008_105249_589,rmse,1,39.161667
1,0030_20151008_105249_589,psnr,1,16.273580
2,0030_20151008_105249_589,ssim,1,0.812035
3,0030_20151008_105249_589,rmse,2,25.583744
4,0030_20151008_105249_589,psnr,2,19.971522
...,...,...,...,...
10855,0043_20160923_122514_781,psnr,4,19.521871
10856,0043_20160923_122514_781,ssim,4,0.928805
10857,0043_20160923_122514_781,rmse,5,30.929737
10858,0043_20160923_122514_781,psnr,5,18.323279


In [58]:
cidxmins = (
    cdf[(cdf["metric"] == "perceptual") | (cdf["metric"] == "rmse")]
    .groupby(["name", "metric"])["score"]
    .idxmin()
)
cbest_scores_min = cdf.loc[cidxmins]
cbest_scores_min
# this is the minimum score for each metric for each image
# unfortunately we want min for mse and perceptual
# and maxes for psnr, ssim

,name,metric,ev,score
3933,0006_20160721_170707_736,rmse,2,5.948025
6009,0006_20160721_175239_909,rmse,4,9.848779
7899,0006_20160721_181525_062,rmse,4,9.309660
2436,0006_20160722_100126_028,rmse,3,8.724895
5181,0006_20160722_100951_201,rmse,3,6.278645
...,...,...,...,...
6138,c483_20150901_173210_443,rmse,2,8.954333
9219,c483_20150901_194029_165,rmse,4,9.003112
8589,c483_20150901_195108_436,rmse,4,12.383640
1221,c483_20150901_214630_530,rmse,3,11.204544


In [59]:
cidxmaxs = (
    cdf[(cdf["metric"] == "psnr") | (cdf["metric"] == "ssim")]
    .groupby(["name", "metric"])["score"]
    .idxmax()
)
cbest_scores_max = cdf.loc[cidxmaxs]
cbest_scores_max

,name,metric,ev,score
3934,0006_20160721_170707_736,psnr,2,32.643347
3935,0006_20160721_170707_736,ssim,2,0.993332
6010,0006_20160721_175239_909,psnr,4,28.263156
6011,0006_20160721_175239_909,ssim,4,0.967660
7900,0006_20160721_181525_062,psnr,4,28.752127
...,...,...,...,...
8591,c483_20150901_195108_436,ssim,4,0.976068
1222,c483_20150901_214630_530,psnr,3,27.142920
1223,c483_20150901_214630_530,ssim,3,0.960959
1255,c483_20150901_221141_886,psnr,4,29.356700


In [60]:
cmin = cbest_scores_min.groupby("metric").describe()["score"][["mean", "std"]]
cmax = cbest_scores_max.groupby("metric").describe()["score"][["mean", "std"]]
pd.concat([cmin, cmax])

,mean,std
metric,,
rmse,10.060834,2.880453
psnr,28.448983,2.632520
ssim,0.976898,0.012301


In [61]:
cdf.groupby(["ev", "metric"]).describe()["score"][["mean", "std"]].to_latex(
    figdir / "conv_choice.tex"
)
cdf.groupby(["ev", "metric"]).describe()["score"][["mean", "std"]]

mean        std
ev metric                      
1  psnr    16.144820   4.237501
   rmse    44.319712  19.385214
   ssim     0.707827   0.226675
2  psnr    20.877057   5.383900
   rmse    27.334192  14.555937
   ssim     0.848401   0.141950
3  psnr    25.248919   3.031392
   rmse    14.784704   5.051738
   ssim     0.943758   0.033221
4  psnr    26.384000   3.998241
   rmse    13.644772   7.042175
   ssim     0.967228   0.020001
5  psnr    19.700461   3.322998
   rmse    28.345435  10.790692
   ssim     0.880723   0.058906